# Vector creation
This code shows the implementation of the vector database for word, sentence and document embeddings.

In [1]:
import jsonlines
path = "lovdata_combined.jsonl"   # used as a sample for multiple documents, in place of the kripos case files
with jsonlines.open(path) as reader:
    data = list(reader)

In [2]:
example = data[6]
example["paragraphs"][-10:]

['Retten har som det fremgår foran, kommet til at oppsigelsen av A er gyldig. Utgangspunktet er likevel at dersom søksmål reises innen de frister som loven setter, kan arbeidstakeren fortsette i stillingen så lenge tvisten pågår. Denne rettigheten gjelder også bedriftens daglige leder, jf. Rt-1999-1998.',
 'I Ot.prp.nr.49 (2004–2005) s. 239 påpekes imidlertid følgende:',
 'Departementet legger til grunn at retten til å fortsette i stillingen, kan være upraktisk å gjennomføre ved oppsigelse av øverste leder i virksomheten. I de tilfellene denne kategorien oppsigelsessak har vært behandlet for domstolene, har retten anerkjent at det må stilles særlige krav til ledere. Departementet legger til grunn at det skal mindre til for at en leder må fratre enn en underordnet ansatt.',
 'På denne bakgrunn finner retten «det urimelig at arbeidsforholdet opprettholdes under sakens behandling». Det vises her til begrunnelsen for den saklige oppsigelsen. Daglig leder har en helt sentral pos

In [3]:
sent = 'Hun dømmes videre til en bot på 25.000 – tjuefemtusen – kroner, subsidiært fengsel i 15 – femten – dager jf. straffeloven § 54.'
sent

'Hun dømmes videre til en bot på 25.000 – tjuefemtusen – kroner, subsidiært fengsel i 15 – femten – dager jf. straffeloven § 54.'

In [4]:
import re

class LovdataPreprocessor:
    def __init__(self):
        self.law_match = re.compile(r"§\s*\d+(-\d+)?")
        self.dash_match = re.compile(r"–\s*\w+\s*–")
        self.num_match = re.compile(r"\d+\.\d+(\.\d+)?")
        
    def preprocess(self, text):
        text = self.law_match.sub("", text)
        text = self.dash_match.sub("", text)
        # illegal_chars = ["[", "]", "{", "}"]
        # text = "".join([c for c in text if c not in illegal_chars])
        text = re.sub(r"\s+", " ", text)
        # use num_match to remove dots in numbers (25.000->25000)
        text = self.num_match.sub(lambda x: x.group().replace(".", ""), text)
        
        return text

preprocessor = LovdataPreprocessor()

In [5]:
example = "250.000 100.000.000"
assert preprocessor.preprocess(example) == "250000 100000000"

## To the embedding space!

In [6]:
import spacy
nlp = spacy.load("nb_core_news_lg")

In [8]:
import sys
sys.path.append("../")
from models.sbert import load, get_centrality, similarity_search
sbert = load()

In [9]:
def spacy_parser(text):
    doc = nlp(text)
    for sent in doc.sents:
        sent = [token for token in sent if not (len(token) == 1 and not token.is_alpha)]
    return doc

In [10]:
example =  '[virksomhet1] har vunnet saken. I samsvar med hovedregelen i tvisteloven § 20-2 første ledd har selskapet da krav på «full erstatning for sine sakskostnader fra motparten». A kan helt eller delvis fritas for erstatningsansvar hvis tungtveiende grunner gjør det rimelig, jf. bestemmelsens tredje ledd. Loven angir selv momenter det ved denne vurdering skal legges særlig vekt på. Disse momentene er imidlertid ikke uttømmende. Retten kan ikke se at tungtveiende grunner tilsier at det er rimelig at A helt eller delvis fritas for erstatningsansvar.'

doc = nlp(example)

In [11]:
for np in doc.noun_chunks:
    # start and end of root token
    root = np.root
    root_start = root.idx
    root_end = root_start + len(root)

    # character idx
    np_start = np.start_char
    np_end = np.end_char
    
    obj = {
        "start": np_start,
        "end": np_end,
        "span": doc.text[np_start:np_end],
        "root": {
            "text": root.text,
            "start": root_start,
            "end": root_end,
            "span": doc.text[root_start:root_end]
        }
    }
    print(obj)

{'start': 1, 'end': 13, 'span': 'virksomhet1]', 'root': {'text': 'virksomhet1', 'start': 1, 'end': 12, 'span': 'virksomhet1'}}
{'start': 25, 'end': 30, 'span': 'saken', 'root': {'text': 'saken', 'start': 25, 'end': 30, 'span': 'saken'}}
{'start': 42, 'end': 79, 'span': 'med hovedregelen i tvisteloven § 20-2', 'root': {'text': 'hovedregelen', 'start': 46, 'end': 58, 'span': 'hovedregelen'}}
{'start': 80, 'end': 91, 'span': 'første ledd', 'root': {'text': 'ledd', 'start': 87, 'end': 91, 'span': 'ledd'}}
{'start': 96, 'end': 105, 'span': 'selskapet', 'root': {'text': 'selskapet', 'start': 96, 'end': 105, 'span': 'selskapet'}}
{'start': 114, 'end': 172, 'span': 'på «full erstatning for sine sakskostnader fra motparten»', 'root': {'text': 'erstatning', 'start': 124, 'end': 134, 'span': 'erstatning'}}
{'start': 174, 'end': 175, 'span': 'A', 'root': {'text': 'A', 'start': 174, 'end': 175, 'span': 'A'}}
{'start': 232, 'end': 252, 'span': 'tungtveiende grunner', 'root': {'text': 'grunner', 'st

In [12]:
doc

[virksomhet1] har vunnet saken. I samsvar med hovedregelen i tvisteloven § 20-2 første ledd har selskapet da krav på «full erstatning for sine sakskostnader fra motparten». A kan helt eller delvis fritas for erstatningsansvar hvis tungtveiende grunner gjør det rimelig, jf. bestemmelsens tredje ledd. Loven angir selv momenter det ved denne vurdering skal legges særlig vekt på. Disse momentene er imidlertid ikke uttømmende. Retten kan ikke se at tungtveiende grunner tilsier at det er rimelig at A helt eller delvis fritas for erstatningsansvar.

In [13]:
for ent in doc.ents:
    print(ent.text, ent.label_)

A PER
A PER


In [16]:
from tqdm import tqdm
from collections import defaultdict

def parse_id(doc):
    return doc["id"].split("/")[-1]

parsed_docs = defaultdict(list)

SUBSET = data[:2]  # change to data for the full file
for doc in tqdm(SUBSET):
    _id = parse_id(doc)
    for para_id, para in enumerate(doc["paragraphs"]):
        para_text = preprocessor.preprocess(para)
        para_doc = spacy_parser(para_text)
        for sent_id, sent in enumerate(para_doc.sents):
            if len(sent) <= 1:
                continue

            # simply add the number of found entities and noun phrases to the sentence
            # this is to weight the matches accordingly
            # e.g. 2x weight for each entity and 1.5x weight for each noun phrase
            num_nps = len(list(sent.noun_chunks))
            num_ents = len(list(sent.ents))

            # check if sent.text has been seen before
            # append the current para_id and sent_id to the existing object
            for obj in parsed_docs[_id]:
                if obj["sent_text"] == sent.text:
                    obj["para_id"].append(para_id)
                    obj["sent_id"].append(sent_id)
                    break
            else:
                parsed_docs[_id].append({
                    "id": f"{_id}_{para_id}_{sent_id}",
                    "para_id": [para_id],
                    "sent_id": [sent_id],
                    "num_nps": num_nps,
                    "num_ents": num_ents,
                    "sent_text": sent.text
                })

100%|██████████| 2/2 [00:05<00:00,  2.92s/it]


In [17]:
import pandas as pd
# convert parsed_docs to dataframe
df = pd.DataFrame([obj for doc in parsed_docs.values() for obj in doc])
df.head()

,id,para_id,sent_id,num_nps,num_ents,sent_text
0,tagd-2021-84238_0_0,[0],[0],2,0,Skattemyndighetene har over tid ført kontroll ...
1,tagd-2021-84238_0_1,[0],[1],3,2,"Det er D (tidligere D), sammen med E, som oppl..."
2,tagd-2021-84238_1_0,[1],[0],6,4,De tre saksøkte er i familie med D; C er hans ...
3,tagd-2021-84238_1_1,[1],[1],3,0,De saksøkte har hatt formelle roller i flere a...
4,tagd-2021-84238_2_0,[2],[0],1,2,Selskapene de var delaktige i skulle ha aktivi...


In [18]:
df.shape

(627, 6)

In [19]:
# compute sbert embeddings
all_sentences = df["sent_text"].tolist()
sbert_embeddings = sbert.encode(all_sentences, show_progress_bar=True)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [20]:
df["sbert_768"] = sbert_embeddings.tolist()

In [21]:
df.head()

,id,para_id,sent_id,num_nps,num_ents,sent_text,sbert_768
0,tagd-2021-84238_0_0,[0],[0],2,0,Skattemyndighetene har over tid ført kontroll ...,"[0.0014578121481463313, -0.41081535816192627, ..."
1,tagd-2021-84238_0_1,[0],[1],3,2,"Det er D (tidligere D), sammen med E, som oppl...","[1.3350529670715332, -0.050390999764204025, 1...."
2,tagd-2021-84238_1_0,[1],[0],6,4,De tre saksøkte er i familie med D; C er hans ...,"[0.9660213589668274, -0.2151397466659546, 1.25..."
3,tagd-2021-84238_1_1,[1],[1],3,0,De saksøkte har hatt formelle roller i flere a...,"[-0.6119834184646606, 0.1516796201467514, 1.15..."
4,tagd-2021-84238_2_0,[2],[0],1,2,Selskapene de var delaktige i skulle ha aktivi...,"[0.7227107882499695, -1.1757646799087524, 1.02..."


In [22]:
# memory usage of dataframe:
df_mb = df.memory_usage(deep=True).sum() / 1024**2
df_mb

4.092142105102539

In [24]:
df.to_pickle("dataframe.pkl")